In [2]:
import torch
from torchdyn.models import NeuralODE
import torch.nn as nn

class ConditionalODEFunc(nn.Module):
    def __init__(self, input_dim, condition_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim + condition_dim, 64),
            nn.Tanh(),
            nn.Linear(64, input_dim)
        )
    
    def forward(self, t, x):
        # This function will be called by the solver
        return self.net(x)

# Define input and condition dimensions
input_dim = 4
condition_dim = 2
func = ConditionalODEFunc(input_dim, condition_dim)

# Create NeuralODE object
neural_ode = NeuralODE(func)

# Initial state `x0`, condition `c`, and time points `t`
x0 = torch.randn(1, input_dim)
c = torch.tensor([[0.5, -0.5]])  # Example condition data
t = torch.linspace(0., 10., 100)

# Define a closure that injects `c` into `x` before passing to the ODE function
def conditional_ode(t, x):
    # Concatenate `x` with `c` to form the input for the neural network
    xc = torch.cat([x, c.expand_as(x)], dim=-1)
    return func(t, xc)

# Use the custom conditional function for integration
solution = neural_ode.odeint(x0, t, conditional_ode)


TypeError: ConditionalODEFunc.forward() got an unexpected keyword argument 'args'